In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from random import randint

from IPython.display import Image
from IPython.core.display import Image, display

import matplotlib.pyplot as plt

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score

%load_ext autoreload
%autoreload 2

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
bs = 32
# Load Data
train_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Training/Normal', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=False)
len(train_dataset.imgs), len(train_dataloader)
train_dataset

Dataset ImageFolder
    Number of datapoints: 256
    Root location: ./No_Image_Preprocessing_Pattern1/Training/Normal
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [4]:
train_dataset.classes

['Normal']

In [5]:
bs = 32
# Load Data
test_dataset_N = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader_N = torch.utils.data.DataLoader(test_dataset_N, batch_size=bs, shuffle=False)

In [6]:
test_dataset_N.classes

['Normal_Same']

In [7]:
bs = 32
# Load Data
test_dataset_N_U = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N_U', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader_N_U = torch.utils.data.DataLoader(test_dataset_N, batch_size=bs, shuffle=False)

In [8]:
test_dataset_N_U.classes

['Normal_Unsame']

In [9]:
bs = 32
# Load Data
test_dataset_A = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/A', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader_A = torch.utils.data.DataLoader(test_dataset_A, batch_size=bs, shuffle=False)

In [10]:
test_dataset_A.classes

['Abnormal']

In [11]:
# Fixed input for debugging
fixed_x, _ = next(iter(train_dataloader))

In [12]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [13]:
class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

In [14]:
class autoencoder(nn.Module):
    def __init__(self,i=1024,o=64):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(512, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            Flatten(),
            nn.Linear(in_features=i, out_features=o),
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=o, out_features=i),
            UnFlatten(),
            nn.ConvTranspose2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [15]:
model= autoencoder()
model.load_state_dict(torch.load('./autoencoder_AE_Thesis_Pattern1_latent64.pth'))
model = model.to(device)

In [16]:
encoder= model.encoder
encoder

Sequential(
  (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
  (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
  (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
  (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (17): ReLU()
  (18): Conv2d(1024,

In [17]:
train_normal_encode = []
for data in train_dataloader:
    img, _ = data
    img = Variable(img).cuda()
    output = encoder(img)
    output = output.cpu()
    output = output.detach().numpy()
    train_normal_encode.append(output)

In [18]:
import numpy as np
train_normal_encode=np.array(train_normal_encode)

In [19]:
a = np.reshape(train_normal_encode, (train_normal_encode.shape[0]*train_normal_encode.shape[1], train_normal_encode.shape[2]))
train_normal_encode=a

In [20]:
test_normal_encode = []
for data_normal in test_dataloader_N:
    img_normal, _ = data_normal
    img_normal = Variable(img_normal).cuda()
    output = encoder(img_normal)
    output = output.cpu()
    output = output.detach().numpy()
    test_normal_encode.append(output)

In [21]:
test_normal_encode=np.array(test_normal_encode)
b = np.reshape(test_normal_encode, (test_normal_encode.shape[0]*test_normal_encode.shape[1], test_normal_encode.shape[2]))
test_normal_encode=b

In [22]:
test_normal_encode_N_U = []
for data_normal_unsame in test_dataloader_N_U:
    img_normal, _ = data_normal
    img_normal = Variable(img_normal).cuda()
    output = encoder(img_normal)
    output = output.cpu()
    output = output.detach().numpy()
    test_normal_encode_N_U.append(output)

In [23]:
test_normal_encode_N_U=np.array(test_normal_encode_N_U)
b = np.reshape(test_normal_encode_N_U, (test_normal_encode_N_U.shape[0]*test_normal_encode_N_U.shape[1], test_normal_encode_N_U.shape[2]))
test_normal_encode_N_U=b

In [24]:
anomaly_encode = []
for data_anomaly in test_dataloader_A:
    img_anomaly, _ = data_anomaly
    img_anomaly = Variable(img_anomaly).cuda()
    output = encoder(img_anomaly)
    output = output.cpu()
    output = output.detach().numpy()
    anomaly_encode.append(output)

In [25]:
anomaly_encode=np.array(anomaly_encode)
b = np.reshape(anomaly_encode, (anomaly_encode.shape[0]*anomaly_encode.shape[1], anomaly_encode.shape[2]))
anomaly_encode=b

In [26]:
train_normal_encode.shape

(256, 64)

In [27]:
# for i in range(train_normal_encode.shape[0]):
#     print(anomaly_encode[i])

In [28]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [29]:
scaler = StandardScaler()
scale_train=scaler.fit_transform(train_normal_encode)
scale_test=scaler.fit_transform(test_normal_encode)
scale_test_unsame=scaler.fit_transform(test_normal_encode_N_U)
scale_anomaly=scaler.fit_transform(anomaly_encode)

In [31]:
# fit the model
nus=[0.001,0.01,0.1,0.3,0.5,0.6,0.7,0.8]
for i in range(len(nus)):
    clf = svm.OneClassSVM(nu=nus[i], kernel="rbf", gamma='auto')
    clf.fit(scale_train)
    y_pred_train = clf.predict(scale_train)
    y_pred_normal = clf.predict(scale_test)
    y_pred_normal_unsame = clf.predict(scale_test_unsame)
    y_pred_anomaly = clf.predict(scale_anomaly)
    n_true_normal = y_pred_normal[y_pred_normal == 1].size
    n_true_normal_unsame = y_pred_normal_unsame[y_pred_normal_unsame == -1].size
    n_true_anomaly = y_pred_anomaly[y_pred_anomaly == -1].size
    n_true_train = y_pred_train[y_pred_train == 1].size
    print('NU:',nus[i],"Kernel: RBF")
    print('TN Train:',n_true_train,'\nTesting:')
    print('TN:',n_true_normal,'TP Anomaly:',n_true_anomaly,'TP Normal Unsame:',n_true_normal_unsame)
    n_error_normal = y_pred_normal[y_pred_normal == -1].size
    n_error_normal_unsame = y_pred_normal_unsame[y_pred_normal_unsame == 1].size
    n_error_anomaly = y_pred_anomaly[y_pred_anomaly == 1].size
    print('FP:',n_error_normal,'FN Anomaly:',n_error_anomaly,'FN Normal Unsame:',n_error_normal_unsame,'\n')

NU: 0.001 Kernel: RBF
TN Train: 224 
Testing:
TN: 86 TP Anomaly: 37 TP Normal Unsame: 44
FP: 42 FN Anomaly: 91 FN Normal Unsame: 84 

NU: 0.01 Kernel: RBF
TN Train: 227 
Testing:
TN: 86 TP Anomaly: 36 TP Normal Unsame: 44
FP: 42 FN Anomaly: 92 FN Normal Unsame: 84 

NU: 0.1 Kernel: RBF
TN Train: 219 
Testing:
TN: 86 TP Anomaly: 36 TP Normal Unsame: 44
FP: 42 FN Anomaly: 92 FN Normal Unsame: 84 

NU: 0.3 Kernel: RBF
TN Train: 182 
Testing:
TN: 79 TP Anomaly: 42 TP Normal Unsame: 52
FP: 49 FN Anomaly: 86 FN Normal Unsame: 76 

NU: 0.5 Kernel: RBF
TN Train: 127 
Testing:
TN: 48 TP Anomaly: 64 TP Normal Unsame: 64
FP: 80 FN Anomaly: 64 FN Normal Unsame: 64 

NU: 0.6 Kernel: RBF
TN Train: 104 
Testing:
TN: 41 TP Anomaly: 72 TP Normal Unsame: 76
FP: 87 FN Anomaly: 56 FN Normal Unsame: 52 

NU: 0.7 Kernel: RBF
TN Train: 77 
Testing:
TN: 30 TP Anomaly: 81 TP Normal Unsame: 88
FP: 98 FN Anomaly: 47 FN Normal Unsame: 40 

NU: 0.8 Kernel: RBF
TN Train: 51 
Testing:
TN: 20 TP Anomaly: 93 TP Normal